In [ ]:
import cv2
import numpy as np

In [2]:
while True:
    #image_ = cv2.imread("images.jpg")
    image = cv2.VideoCapture(0)
    _,image_= image.read()
    model = cv2.dnn.readNet("yolov3.weights","yolov3.cfg")
    with open ("coco.names","r") as f:
        names = f.read().split("\n") 

    #print(names)
    layer_names = model.getLayerNames()
    output_layers = [layer_names[i[0]-1] for i in model.getUnconnectedOutLayers()]
    colours = np.random.uniform(0,255,size = (len(names),3))
    #image = cv2.resize(image ,(416,416))
    #print(image_.shape)
    hh,ww,_ = image_.shape
    #converting image to binary object
    blob_image = cv2.dnn.blobFromImage(image_,
                                      (1 / 255),
                                      (416,416),
                                      (False,False,False),
                                       crop = False)
    model.setInput(blob_image)
    output = model.forward(output_layers)
    """
    print(output)  
    print(output[0])
    """
        # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in output:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * ww)
                center_y = int(detection[1] * hh)
                w = int(detection[2] * ww)
                h = int(detection[3] * hh)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(names[class_ids[i]])
            color = colours[i]
            cv2.rectangle(image_, (x, y), (x + w, y + h), color, 2)
            cv2.putText(image_, label, (x, y + 30), font, 3, color, 3)
    cv2.imshow("Image", image_)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    image.release()                

KeyboardInterrupt: 